# 4. MCQ-Question

3장에서는 BERT와 GPT를 사용하여 True/False 문제를 생성해보았습니다. 4장에서는 T5모델로 Multiple-Choice Question(MCQ) 문제를 생성해보도록 하겠습니다. 

4.1절에서는 모델에 인풋으로 사용할 에세이 데이터를 불러오겠습니다. 그리고 4.2절에서는 Question을 Generation하는 클래스를, 4.3절에서는 Question Answer을 Eval하는 클래스를 정의하도록 하겠습니다. 마지막으로 4.4절에서 실제 에세이 데이터를 넣어 MCQ문제를 생성해보도록 하겠습니다.

## 4.1 데이터셋 다운로드

2.?절에서 나온 코드를 활용하여 올바른 표현으로 수정된 데이터셋을 불러오도록 하겠습니다. 이 데이터는 MCQ 문제를 만들 지문으로 사용됩니다.

In [18]:
import pickle
file_name = "C:/Users/minji/OneDrive - 고려대학교/PseudoLab/tutorial/articles/corrected_essays.pkl"
open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

In [19]:
open_file
## essay 예시 보여주기

<_io.BufferedReader name='C:/Users/minji/OneDrive - 고려대학교/PseudoLab/tutorial/articles/corrected_essays.pkl'>

In [6]:
# https://github.com/AMontgomerie/question_generator/tree/master/articles
with open('C:/Users/minji/OneDrive - 고려대학교/PseudoLab/tutorial/articles/indian_matchmaking.txt', 'r') as a:
    article = a.read()

4장에서 사용할 패키지들을 import 해보겠습니다. 각 환경에 따라 패키지가 없거나 버전 차이가 있을 수 있습니다. 아래 코드를 통해 패키지를 설치하거나 버전을 맞추어 사용하시기 바랍니다.

`random`은 난수를 생성하기 위한 패키지이고, `numpy`는 수치연산에 사용하는 패키지입니다. `json`은 파일 형식을 위한 패키지이며 `re`는 정규 표현식을 지원하는 패키지입니다. `torch`는 모델 구축 프레임워크이고, `spacy`는 자연어 처리를 쉽게 할 수 있도록 도와주는 패키지이며 자체적으로 배포한 언어모델이 `en_core_web_sm` 패키지입니다. 또한 다양한 토큰화 기법과 모델들이 내장되어 있는 `transformers` 패키지도 있습니다.

In [9]:
# requirements
# !pip install transformers==4.1.1
# !pip install spacy==2.3.1
# !pip install sentencepiece==0.1.94
# !pip install torch
# !python -m spacy download en_core_web_sm

In [13]:
# import os
# import sys
# import math
import random
import numpy as np
import json
import re
import torch
import spacy
import en_core_web_sm
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    AutoModelForSequenceClassification,
)

본격적인 모델링에 앞서 실험 환경에서 GPU가 사용 가능한지 확인할 수 있습니다. GPU가 없다면 CPU를 사용해도 가능합니다.

In [12]:
print('cuda' if torch.cuda.is_available() else 'cpu')

cuda


## 4.2 Question Generation 클래스 정의

4장에서는 huggingface에서 제공하는 API을 불러와 사용하는 법을 설명합니다.

In [2]:
class QuestionGenerator:
    def __init__(self, model_dir=None):

        QG_PRETRAINED = "iarfmoose/t5-base-question-generator"
        self.ANSWER_TOKEN = "<answer>"
        self.CONTEXT_TOKEN = "<context>"
        self.SEQ_LENGTH = 512

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.qg_tokenizer = AutoTokenizer.from_pretrained(QG_PRETRAINED, use_fast=False)
        self.qg_model = AutoModelForSeq2SeqLM.from_pretrained(QG_PRETRAINED)
        self.qg_model.to(self.device)

        self.qa_evaluator = QAEvaluator(model_dir)

    def generate(
        self, article, use_evaluator=True, num_questions=None, answer_style="all"
    ):

        print("Generating questions...\n")

        qg_inputs, qg_answers = self.generate_qg_inputs(article, answer_style)
        generated_questions = self.generate_questions_from_inputs(qg_inputs)

        message = "{} questions doesn't match {} answers".format(
            len(generated_questions), len(qg_answers)
        )
        assert len(generated_questions) == len(qg_answers), message

        if use_evaluator:

            print("Evaluating QA pairs...\n")

            encoded_qa_pairs = self.qa_evaluator.encode_qa_pairs(
                generated_questions, qg_answers
            )
            scores = self.qa_evaluator.get_scores(encoded_qa_pairs)
            if num_questions:
                qa_list = self._get_ranked_qa_pairs(
                    generated_questions, qg_answers, scores, num_questions
                )
            else:
                qa_list = self._get_ranked_qa_pairs(
                    generated_questions, qg_answers, scores
                )

        else:
            print("Skipping evaluation step.\n")
            qa_list = self._get_all_qa_pairs(generated_questions, qg_answers)

        return qa_list

    def generate_qg_inputs(self, text, answer_style):

        VALID_ANSWER_STYLES = ["all", "sentences", "multiple_choice"]

        if answer_style not in VALID_ANSWER_STYLES:
            raise ValueError(
                "Invalid answer style {}. Please choose from {}".format(
                    answer_style, VALID_ANSWER_STYLES
                )
            )

        inputs = []
        answers = []

        if answer_style == "sentences" or answer_style == "all":
            segments = self._split_into_segments(text)  # 여러 문장씩 합쳐서 segments로 바꿈
            for segment in segments:
                sentences = self._split_text(segment)   # 줄단위를 문장단위로 바꿈
                prepped_inputs, prepped_answers = self._prepare_qg_inputs(
                    sentences, segment  
                )
                inputs.extend(prepped_inputs)
                answers.extend(prepped_answers)

        if answer_style == "multiple_choice" or answer_style == "all":
            sentences = self._split_text(text)
            prepped_inputs, prepped_answers = self._prepare_qg_inputs_MC(sentences)
            inputs.extend(prepped_inputs)
            answers.extend(prepped_answers)

        return inputs, answers

    def generate_questions_from_inputs(self, qg_inputs):
        generated_questions = []

        for qg_input in qg_inputs:
            question = self._generate_question(qg_input)
            generated_questions.append(question)

        return generated_questions

    def _split_text(self, text):
        MAX_SENTENCE_LEN = 128

        sentences = re.findall(".*?[.!\?]", text)  # 다시 한문장씩 잘라줌

        cut_sentences = []
        for sentence in sentences:
            if len(sentence) > MAX_SENTENCE_LEN:   
                cut_sentences.extend(re.split("[,;:)]", sentence))      # 길면 문장을 구 단위로 또 나눠서 합침, 짧은 문장은 없어짐
        # temporary solution to remove useless post-quote sentence fragments
        cut_sentences = [s for s in sentences if len(s.split(" ")) > 5]   # 5단어 이상인 문장들만 합침. 왜 두번??????
        sentences = sentences + cut_sentences

        return list(set([s.strip(" ") for s in sentences]))  # 한문장씩으로 잘라줌

    def _split_into_segments(self, text):
        MAX_TOKENS = 490

        paragraphs = text.split("\n")
        tokenized_paragraphs = [
            self.qg_tokenizer(p)["input_ids"] for p in paragraphs if len(p) > 0    # p가 한 줄 , qg_tokenizer(p) = input_ids와 attention_mask가 dict 형태로
        ]  # 한문장씩 list 형태

        segments = []
        while len(tokenized_paragraphs) > 0:
            segment = []
            while len(segment) < MAX_TOKENS and len(tokenized_paragraphs) > 0:
                paragraph = tokenized_paragraphs.pop(0)
                segment.extend(paragraph)
            segments.append(segment)  # MAX_TOKENS 맞춰서 여러 문장을 합쳐 segment 
        return [self.qg_tokenizer.decode(s) for s in segments] # 다시 token이 실제 단어로

    def _prepare_qg_inputs(self, sentences, text):
        inputs = []
        answers = []

        for sentence in sentences:
            qg_input = "{} {} {} {}".format(
                self.ANSWER_TOKEN, sentence, self.CONTEXT_TOKEN, text
            )
            inputs.append(qg_input)
            answers.append(sentence)

        return inputs, answers  # 한문장씩 답이랑 paragraph 나눈거랑 연결

    def _prepare_qg_inputs_MC(self, sentences):

        spacy_nlp = en_core_web_sm.load()
        docs = list(spacy_nlp.pipe(sentences, disable=["parser"]))
        inputs_from_text = []
        answers_from_text = []

        for i in range(len(sentences)):
            entities = docs[i].ents
            if entities:
                for entity in entities:
                    qg_input = "{} {} {} {}".format(
                        self.ANSWER_TOKEN, entity, self.CONTEXT_TOKEN, sentences[i]
                    )
                    answers = self._get_MC_answers(entity, docs)
                    inputs_from_text.append(qg_input)
                    answers_from_text.append(answers)

        return inputs_from_text, answers_from_text

    def _get_MC_answers(self, correct_answer, docs):

        entities = []
        for doc in docs:
            entities.extend([{"text": e.text, "label_": e.label_} for e in doc.ents])

        # remove duplicate elements
        entities_json = [json.dumps(kv) for kv in entities]
        pool = set(entities_json)
        num_choices = (
            min(4, len(pool)) - 1
        )  # -1 because we already have the correct answer

        # add the correct answer
        final_choices = []
        correct_label = correct_answer.label_
        final_choices.append({"answer": correct_answer.text, "correct": True})
        pool.remove(
            json.dumps({"text": correct_answer.text, "label_": correct_answer.label_})
        )

        # find answers with the same NER label
        matches = [e for e in pool if correct_label in e]

        # if we don't have enough then add some other random answers
        if len(matches) < num_choices:
            choices = matches
            pool = pool.difference(set(choices))
            choices.extend(random.sample(pool, num_choices - len(choices)))
        else:
            choices = random.sample(matches, num_choices)

        choices = [json.loads(s) for s in choices]
        for choice in choices:
            final_choices.append({"answer": choice["text"], "correct": False})
        random.shuffle(final_choices)
        return final_choices

    def _generate_question(self, qg_input):
        self.qg_model.eval()
        encoded_input = self._encode_qg_input(qg_input)
        with torch.no_grad():
            output = self.qg_model.generate(input_ids=encoded_input["input_ids"])
        question = self.qg_tokenizer.decode(output[0], skip_special_tokens=True)
        return question

    def _encode_qg_input(self, qg_input):
        return self.qg_tokenizer(
            qg_input,
            padding='max_length',
            max_length=self.SEQ_LENGTH,
            truncation=True,
            return_tensors="pt",
        ).to(self.device)

    def _get_ranked_qa_pairs(
        self, generated_questions, qg_answers, scores, num_questions=10
    ):
        if num_questions > len(scores):
            num_questions = len(scores)
            print(
                "\nWas only able to generate {} questions. For more questions, please input a longer text.".format(
                    num_questions
                )
            )

        qa_list = []
        for i in range(num_questions):
            index = scores[i]
            qa = self._make_dict(
                generated_questions[index].split("?")[0] + "?", qg_answers[index]
            )
            qa_list.append(qa)
        return qa_list

    def _get_all_qa_pairs(self, generated_questions, qg_answers):
        qa_list = []
        for i in range(len(generated_questions)):
            qa = self._make_dict(
                generated_questions[i].split("?")[0] + "?", qg_answers[i]
            )
            qa_list.append(qa)
        return qa_list

    def _make_dict(self, question, answer):
        qa = {}
        qa["question"] = question
        qa["answer"] = answer
        return qa

## 4.3 Question Answer Eval 클래스 정의

In [3]:
class QAEvaluator:
    def __init__(self, model_dir=None):

        QAE_PRETRAINED = "iarfmoose/bert-base-cased-qa-evaluator"
        self.SEQ_LENGTH = 512

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.qae_tokenizer = AutoTokenizer.from_pretrained(QAE_PRETRAINED)
        self.qae_model = AutoModelForSequenceClassification.from_pretrained(
            QAE_PRETRAINED
        )
        self.qae_model.to(self.device)

    def encode_qa_pairs(self, questions, answers):
        encoded_pairs = []
        for i in range(len(questions)):
            encoded_qa = self._encode_qa(questions[i], answers[i])
            encoded_pairs.append(encoded_qa.to(self.device))
        return encoded_pairs

    def get_scores(self, encoded_qa_pairs):
        scores = {}
        self.qae_model.eval()
        with torch.no_grad():
            for i in range(len(encoded_qa_pairs)):
                scores[i] = self._evaluate_qa(encoded_qa_pairs[i])

        return [
            k for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)
        ]

    def _encode_qa(self, question, answer):
        if type(answer) is list:
            for a in answer:
                if a["correct"]:
                    correct_answer = a["answer"]
        else:
            correct_answer = answer
        return self.qae_tokenizer(
            text=question,
            text_pair=correct_answer,
            padding="max_length",
            max_length=self.SEQ_LENGTH,
            truncation=True,
            return_tensors="pt",
        )

    def _evaluate_qa(self, encoded_qa_pair):
        output = self.qae_model(**encoded_qa_pair)
        return output[0][0][1]

생성된 문제를 보기 좋게 출력하기 위해 `print_qa`함수를 정의합니다.

In [4]:
def print_qa(qa_list, show_answers=True):
    for i in range(len(qa_list)):
        space = " " * int(np.where(i < 9, 3, 4))  # wider space for 2 digit q nums

        print("{}) Q: {}".format(i + 1, qa_list[i]["question"]))

        answer = qa_list[i]["answer"]

        # print a list of multiple choice answers
        if type(answer) is list:

            if show_answers:
                print(
                    "{}A: 1.".format(space),
                    answer[0]["answer"],
                    np.where(answer[0]["correct"], "(correct)", ""),
                )
                for j in range(1, len(answer)):
                    print(
                        "{}{}.".format(space + "   ", j + 1),
                        answer[j]["answer"],
                        np.where(answer[j]["correct"] == True, "(correct)", ""),
                    )

            else:
                print("{}A: 1.".format(space), answer[0]["answer"])
                for j in range(1, len(answer)):
                    print("{}{}.".format(space + "   ", j + 1), answer[j]["answer"])
            print("")

        # print full sentence answers
        else:
            if show_answers:
                print("{}A:".format(space), answer, "\n")

## 4.4 MCQ Question 생성

위에서 정의한 클래스를 불러 `qg`모델을 구축합니다. 

In [7]:
qg = QuestionGenerator()

Downloading: 100%|██████████| 892M/892M [00:59<00:00, 14.9MB/s]
Some weights of the model checkpoint at iarfmoose/t5-base-question-generator were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Downloading: 100%|██████████| 482/482 [00:00<00:00, 483kB/s]
Downloading: 100%|██████████| 213k/213k [00:00<00:00, 362kB/s] 
Downloading: 100%|██████████| 112/112 [00:00<?, ?B/s] 
Downloading: 100%|██████████| 49.0/49.0 [00:00<00:00, 49.

`qg`모델의 메인 함수인 `generate`에서 문제를 생성하여 list 형태로 `qa_list`에 저장합니다. 파라미터인 `num_questions`로 생성할 문제 수를 지정할 수 있습니다. 그리고 `print_qa`함수로 출력하여 최종 문제지를 생성할 수 있습니다.

In [11]:
qa_list = qg.generate(
    article, 
    num_questions=10, 
    answer_style='multiple_choice'
)
print_qa(qa_list)

Generating questions...

Evaluating QA pairs...

1) Q: How long did Indian Matchmaking last?
   A: 1. the years 
      2. nearly two weeks (correct)
      3. 50s 
      4. 30s 

2) Q: Who is the "aunt" to her clients?
   A: 1. Tinder 
      2. Anna MM Vetticad 
      3. Ms Taparia (correct)
      4. Kiran Lamba Jha 

3) Q: How old are they?
   A: 1. the years 
      2. 50s 
      3. nearly two weeks 
      4. 30s (correct)

4) Q: In what country is Sima Taparia trying to find suitable matches?
   A: 1. Taparia 
      2. US (correct)
      3. India 
      4. Mumbai 

5) Q: Who is the mother of a child?
   A: 1. Ms Taparia's 
      2. Kiran Lamba Jha 
      3. Ms Taparia (correct)
      4. Sima Taparia 

6) Q: How many episodes of the docuseries are there?
   A: 1. One 
      2. eight (correct)
      3. 150 
      4. thousands 

7) Q: How many items of clothing do she have?
   A: 1. hundreds 
      2. Hundreds 
      3. dozens (correct)
      4. 1 

8) Q: How much money is at stake in ma

## ???? 모델 학습(이후에 삭제)

In [6]:
import os
import sys
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import spacy
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

In [7]:
# https://huggingface.co/models?filter=pytorch
PRETRAINED_MODEL = 'valhalla/t5-small-qg-prepend'
DIR = "question_generator/"
BATCH_SIZE = 6
SEQ_LENGTH = 512

tokenizer = T5Tokenizer.from_pretrained(PRETRAINED_MODEL)
tokenizer.add_special_tokens(
    {'additional_special_tokens': ['<answer>', '<context>']}
)

Downloading: 100%|██████████| 792k/792k [00:01<00:00, 670kB/s]
Downloading: 100%|██████████| 31.0/31.0 [00:00<00:00, 31.0kB/s]
Downloading: 100%|██████████| 65.0/65.0 [00:00<00:00, 32.5kB/s]
Downloading: 100%|██████████| 90.0/90.0 [00:00<00:00, 90.0kB/s]


2

In [8]:
class QGDataset(Dataset):
    def __init__(self, csv):
        self.df = pd.read_csv(csv, engine='python')[:18]

    def __len__(self):
         return len(self.df)

    def __getitem__(self, idx):   
        if torch.is_tensor(idx):
            idx = idx.tolist()
        row = self.df.iloc[idx, 1:]       

        encoded_text = tokenizer(
            row['text'], 
            pad_to_max_length=True, 
            max_length=SEQ_LENGTH,
            truncation=True,
            return_tensors="pt"
        )
        encoded_text['input_ids'] = torch.squeeze(encoded_text['input_ids'])
        encoded_text['attention_mask'] = torch.squeeze(encoded_text['attention_mask'])

        encoded_question = tokenizer(
            row['question'],
            pad_to_max_length=True,
            max_length=SEQ_LENGTH,
            truncation=True,
            return_tensors='pt'
        )
        encoded_question['input_ids'] = torch.squeeze(encoded_question['input_ids'])

        return (encoded_text.to(device), encoded_question.to(device))

In [10]:
train_set = QGDataset(os.path.join(DIR, 'C:/Users/minji/OneDrive - 고려대학교/PseudoLab/tutorial/articles/qg_train_small.csv'))
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
valid_set = QGDataset(os.path.join(DIR, 'C:/Users/minji/OneDrive - 고려대학교/PseudoLab/tutorial/articles/qg_valid_small.csv')) 
valid_loader = DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=False)

In [11]:
LR = 0.001
EPOCHS = 20
LOG_INTERVAL = 5000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [12]:
config = T5Config(decoder_start_token_id=tokenizer.pad_token_id)
model = T5ForConditionalGeneration(config).from_pretrained(PRETRAINED_MODEL)
model.resize_token_embeddings(len(tokenizer)) # to account for new special tokens
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

Downloading: 100%|██████████| 1.02k/1.02k [00:00<00:00, 508kB/s]
Downloading: 100%|██████████| 242M/242M [00:02<00:00, 84.4MB/s]
Some weights of the model checkpoint at valhalla/t5-small-qg-prepend were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
def train(epoch, best_val_loss):
    model.train()
    total_loss = 0.
    for batch_index, batch in enumerate(train_loader):
        print(batch_index+1, '/', len(train_loader))
        data, target = batch
        optimizer.zero_grad()
        masked_labels = mask_label_padding(target['input_ids'])
        output = model(
            input_ids=data['input_ids'],
            attention_mask=data['attention_mask'],
            labels=masked_labels
        )
        loss = output[0]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch_index % LOG_INTERVAL == 0 and batch_index > 0:
            cur_loss = total_loss / LOG_INTERVAL
            print('| epoch {:3d} | ' 
                  '{:5d}/{:5d} batches | '
                  'loss {:5.2f}'.format(
                    epoch, 
                    batch_index, len(train_loader), 
                    cur_loss))
            save(
                TEMP_SAVE_PATH,
                epoch, 
                model.state_dict(), 
                optimizer.state_dict(), 
                best_val_loss
            )
            total_loss = 0

def evaluate(eval_model, data_loader):
    eval_model.eval()
    total_loss = 0.
    with torch.no_grad():
        for batch_index, batch in enumerate(data_loader):
            print(batch_index+1, '/', len(data_loader))
            data, target = batch
            masked_labels = mask_label_padding(target['input_ids'])
            output = eval_model(
                input_ids=data['input_ids'],
                attention_mask=data['attention_mask'],
                labels=masked_labels
            )
            total_loss += output[0].item()
    return total_loss / len(data_loader)

def mask_label_padding(labels):
    MASK_ID = -100
    labels[labels==tokenizer.pad_token_id] = MASK_ID
    return labels

def save(path, epoch, model_state_dict, optimizer_state_dict, loss):
    torch.save({
            'epoch': epoch,
            'model_state_dict': model_state_dict,
            'optimizer_state_dict': optimizer_state_dict,
            'best_loss': loss,
            }, path)

def load(path):
    return torch.load(path)

def print_line():
    LINE_WIDTH = 60
    print('-' * LINE_WIDTH)

In [16]:
best_val_loss = float("inf")
best_model = None

val_loss = evaluate(model, valid_loader)
print_line()
print('| Before training | valid loss {:5.2f}'.format(
    val_loss)
)
print_line()

for epoch in range(1, EPOCHS + 1):

    train(epoch, best_val_loss)
    val_loss = evaluate(model, valid_loader)
    print_line()
    print('| end of epoch {:3d} | valid loss {:5.2f}'.format(
        epoch,
        val_loss)
    )
    print_line()

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model
        save(
             f"C:/Users/minji/OneDrive - 고려대학교/PseudoLab/tutorial/articles/qg_pretrained_t5_model_trained_{epoch}.pth",
             epoch, 
             model.state_dict(), 
             optimizer.state_dict(), 
             best_val_loss
        )
        print("| Model saved.")
        print_line()

1 / 2
2 / 2
------------------------------------------------------------
| Before training | valid loss  3.98
------------------------------------------------------------
1 / 3
2 / 3
3 / 3
1 / 2
2 / 2
------------------------------------------------------------
| end of epoch   1 | valid loss  3.97
------------------------------------------------------------
| Model saved.
------------------------------------------------------------
1 / 3
2 / 3
3 / 3
1 / 2
2 / 2
------------------------------------------------------------
| end of epoch   2 | valid loss  3.96
------------------------------------------------------------
| Model saved.
------------------------------------------------------------
1 / 3
2 / 3
3 / 3
1 / 2
2 / 2
------------------------------------------------------------
| end of epoch   3 | valid loss  3.95
------------------------------------------------------------
| Model saved.
------------------------------------------------------------
1 / 3
2 / 3
3 / 3
1 / 2
2 / 2
-